In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import modules.utils as Utils
from modules.regex_patterns import reviews_paths
from collections import Counter
from underthesea import word_tokenize

### 14. Load dữ liệu `standard_data.csv`

In [3]:
raw_data = pd.read_csv("./data/preprocessing_data.csv")

In [4]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,NaN,lần đầu tại ưng ý vải mát lót đùi thun sớm đón...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,NaN,nhanh xịn xò xong chủ động nhắn tin xác nhận g...,giao_hàng_nhanh chất_vải cẩn_thận xịn_xò túi_z...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,NaN,vừa vặn chất lượng giá rẻ vầy tội mu hả các bác,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,NaN,sản phẩm chất lượng đóng gói kĩ nhiệt tình ủng...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,NaN,chất lượng sản phẩm tuyệt vời nhanh ủng hộ dài...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,1,NaN,nơ xinh đúng chất công chúa đẹp,NaN
220025,"Rất dễ thương, đẹp",1,NaN,dễ thương đẹp,dễ_thương
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,NaN,đẹp mọi ơi học,mọi_người đẹp_lắm_luôn
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,NaN,món đủ vừa rẻ vừa chất lượng thích ủng hộ tiếp,ủng_hộ chất_lượng giao_đủ


In [5]:
raw_data = raw_data.replace(np.nan, "")
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu tại ưng ý vải mát lót đùi thun sớm đón...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,nhanh xịn xò xong chủ động nhắn tin xác nhận g...,giao_hàng_nhanh chất_vải cẩn_thận xịn_xò túi_z...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vừa vặn chất lượng giá rẻ vầy tội mu hả các bác,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ nhiệt tình ủng...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời nhanh ủng hộ dài...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,
220025,"Rất dễ thương, đẹp",1,,dễ thương đẹp,dễ_thương
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp mọi ơi học,mọi_người đẹp_lắm_luôn
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,món đủ vừa rẻ vừa chất lượng thích ủng hộ tiếp,ủng_hộ chất_lượng giao_đủ


### 15. Xóa các `boost_words` khỏi `processing_comment`

In [6]:
raw_data['processing_comment'] = raw_data.apply(lambda row: Utils.removeBoostWord(row['processing_comment'], row['boost_words']), axis=1)

In [7]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu tại vải mát lót đùi thun sớm mỗi sản b...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,nhanh xong chủ động nhắn tin xác nhận gói mỗi ...,giao_hàng_nhanh chất_vải cẩn_thận xịn_xò túi_z...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vầy tội mu hả các bác,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,kĩ nhiệt tình lần cảm ơn,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,nhanh dài dài,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,
220025,"Rất dễ thương, đẹp",1,,đẹp,dễ_thương
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp mọi ơi học,mọi_người đẹp_lắm_luôn
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,món đủ vừa rẻ vừa thích tiếp,ủng_hộ chất_lượng giao_đủ


### 16, Tokenize bằng `underthesea`

In [8]:
def undertheseaTokenize(pText):
    return " ".join([re.sub(" ", "_", word) for word in word_tokenize(str(pText))])

In [9]:
raw_data['tokenize'] = raw_data['processing_comment'].apply(lambda comment: undertheseaTokenize(comment))

In [10]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words,tokenize
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu tại vải mát lót đùi thun sớm mỗi sản b...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...,lần đầu tại vải mát lót đùi thun sớm mỗi sản b...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,nhanh xong chủ động nhắn tin xác nhận gói mỗi ...,giao_hàng_nhanh chất_vải cẩn_thận xịn_xò túi_z...,nhanh xong chủ_động nhắn_tin xác_nhận gói mỗi ...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vầy tội mu hả các bác,giá_rẻ chất_lượng vừa_vặn,vầy tội mu hả các bác
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,kĩ nhiệt tình lần cảm ơn,ủng_hộ đóng_gói chất_lượng sản_phẩm,kĩ nhiệt_tình lần cảm_ơn
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,nhanh dài dài,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...,nhanh dài_dài
...,...,...,...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,,nơ xinh đúng chất công_chúa đẹp
220025,"Rất dễ thương, đẹp",1,,đẹp,dễ_thương,đẹp
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp mọi ơi học,mọi_người đẹp_lắm_luôn,đẹp mọi ơi học
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,món đủ vừa rẻ vừa thích tiếp,ủng_hộ chất_lượng giao_đủ,món đủ vừa rẻ vừa thích tiếp


### 17. Kết hợp 3 cột `tokenize`, `boost_words` và `emoji` lại thành một cột duy nhất là `standard_comment`

In [11]:
raw_data['standard_comment'] = raw_data.apply(lambda row: Utils.genStandardComment(row['tokenize'], row['boost_words'], row['emoji']), axis=1)

In [12]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words,tokenize,standard_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu tại vải mát lót đùi thun sớm mỗi sản b...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...,lần đầu tại vải mát lót đùi thun sớm mỗi sản b...,lần đầu tại vải mát lót đùi thun sớm mỗi sản b...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,nhanh xong chủ động nhắn tin xác nhận gói mỗi ...,giao_hàng_nhanh chất_vải cẩn_thận xịn_xò túi_z...,nhanh xong chủ_động nhắn_tin xác_nhận gói mỗi ...,nhanh xong chủ_động nhắn_tin xác_nhận gói mỗi ...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vầy tội mu hả các bác,giá_rẻ chất_lượng vừa_vặn,vầy tội mu hả các bác,vầy tội mu hả các bác giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,kĩ nhiệt tình lần cảm ơn,ủng_hộ đóng_gói chất_lượng sản_phẩm,kĩ nhiệt_tình lần cảm_ơn,kĩ nhiệt_tình lần cảm_ơn ủng_hộ đóng_gói chất_...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,nhanh dài dài,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...,nhanh dài_dài,nhanh dài_dài giao_hàng_nhanh ủng_hộ tuyệt_vời...
...,...,...,...,...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,,nơ xinh đúng chất công_chúa đẹp,nơ xinh đúng chất công_chúa đẹp
220025,"Rất dễ thương, đẹp",1,,đẹp,dễ_thương,đẹp,đẹp dễ_thương
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp mọi ơi học,mọi_người đẹp_lắm_luôn,đẹp mọi ơi học,đẹp mọi ơi học mọi_người đẹp_lắm_luôn
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,món đủ vừa rẻ vừa thích tiếp,ủng_hộ chất_lượng giao_đủ,món đủ vừa rẻ vừa thích tiếp,món đủ vừa rẻ vừa thích tiếp ủng_hộ chất_lượng...


### 18. Xóa các cột thừa đi

In [13]:
raw_data = raw_data.drop(columns=['emoji', 'processing_comment', 'boost_words', 'tokenize'])

In [14]:
raw_data = raw_data.reindex(columns=['raw_comment', 'standard_comment', 'label'])

In [15]:
raw_data

,raw_comment,standard_comment,label
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",lần đầu tại vải mát lót đùi thun sớm mỗi sản b...,1
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",nhanh xong chủ_động nhắn_tin xác_nhận gói mỗi ...,1
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",vầy tội mu hả các bác giá_rẻ chất_lượng vừa_vặn,1
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,kĩ nhiệt_tình lần cảm_ơn ủng_hộ đóng_gói chất_...,1
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,nhanh dài_dài giao_hàng_nhanh ủng_hộ tuyệt_vời...,1
...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,nơ xinh đúng chất công_chúa đẹp,1
220025,"Rất dễ thương, đẹp",đẹp dễ_thương,1
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,đẹp mọi ơi học mọi_người đẹp_lắm_luôn,1
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,món đủ vừa rẻ vừa thích tiếp ủng_hộ chất_lượng...,1


In [16]:
raw_data = raw_data[raw_data['standard_comment'] != ""]

In [17]:
raw_data

,raw_comment,standard_comment,label
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",lần đầu tại vải mát lót đùi thun sớm mỗi sản b...,1
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",nhanh xong chủ_động nhắn_tin xác_nhận gói mỗi ...,1
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",vầy tội mu hả các bác giá_rẻ chất_lượng vừa_vặn,1
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,kĩ nhiệt_tình lần cảm_ơn ủng_hộ đóng_gói chất_...,1
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,nhanh dài_dài giao_hàng_nhanh ủng_hộ tuyệt_vời...,1
...,...,...,...
220024,Nơ xinh đúng chất công chúa đẹpppppp,nơ xinh đúng chất công_chúa đẹp,1
220025,"Rất dễ thương, đẹp",đẹp dễ_thương,1
220026,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,đẹp mọi ơi học mọi_người đẹp_lắm_luôn,1
220027,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,món đủ vừa rẻ vừa thích tiếp ủng_hộ chất_lượng...,1


### 19. Ghi ra file

In [20]:
raw_data.to_csv("./data/standard_data.csv", index=False)